In [8]:
import requests
import pandas as pd

api_key = "PQyGMdEdKnn5JJKJEjROp3Wij9yjgEmC"

tickers = ["MSFT"]

# define time periods for financial statement data
statement_type = "pl"
time_periods = ["Q1","Q2","Q3","Q4"]
year_start = 2013
year_end = 2018


In [14]:
sim_ids = []
for ticker in tickers:

    request_url = f'https://simfin.com/api/v1/info/find-id/ticker/{ticker}?api-key={api_key}'
    content = requests.get(request_url)
    data = content.json()

    if "error" in data or len(data) < 1:
        sim_ids.append(None)
    else:
        sim_ids.append(data[0]['simId'])

print(sim_ids)

[59265]


In [10]:
# define time periods for financial statement data
statement_type = "pl"
time_periods = ["Q1","Q2","Q3","Q4"]
year_start = 2013
year_end = 2018


In [11]:
data = {}
for idx, sim_id in enumerate(sim_ids):
    d = data[tickers[idx]] = {"Line Item": []}
    if sim_id is not None:
        for year in range(year_start, year_end + 1):
            for time_period in time_periods:

                period_identifier = time_period + "-" + str(year)

                if period_identifier not in d:
                    d[period_identifier] = []

                request_url = f'https://simfin.com/api/v1/companies/id/{sim_id}/statements/standardised?stype={statement_type}&fyear={year}&ptype={time_period}&api-key={api_key}'

                content = requests.get(request_url)
                statement_data = content.json()

                # collect line item names once, they are the same for all companies with the standardised data
                if len(d['Line Item']) == 0:
                    d['Line Item'] = [x['standardisedName'] for x in statement_data['values']]

                if 'values' in statement_data:
                    for item in statement_data['values']:
                        d[period_identifier].append(item['valueChosen'])
                else:
                    # no data found for time period
                    d[period_identifier] = [None for _ in d['Line Item']]


In [13]:
df = pd.DataFrame(data=d)
df

,Line Item,Q1-2013,Q2-2013,Q3-2013,Q4-2013,Q1-2014,Q2-2014,Q3-2014,Q4-2014,Q1-2015,...,Q3-2016,Q4-2016,Q1-2017,Q2-2017,Q3-2017,Q4-2017,Q1-2018,Q2-2018,Q3-2018,Q4-2018
0,Revenue,16008000000,21456000000,20489000000,1.989600e+10,18529000000,24519000000,20403000000,2.338200e+10,23201000000,...,20531000000,2.644800e+10,21928000000,25826000000,23212000000,2.560500e+10,24538000000,28918000000,26819000000,3.008500e+10
1,Sales & Services Revenue,None,None,None,NaN,None,None,None,8.683300e+10,None,...,20531000000,-6.470600e+10,21928000000,25826000000,23212000000,-7.096600e+10,None,28918000000,26819000000,-5.573700e+10
2,Financing Revenue,None,None,None,NaN,None,None,None,NaN,None,...,None,NaN,None,None,None,NaN,None,None,None,NaN
3,Other Revenue,None,None,None,NaN,None,None,None,NaN,None,...,None,NaN,None,None,None,NaN,None,None,None,NaN
4,Cost of revenue,-4168000000,-5692000000,-4787000000,-5.738000e+09,-5145000000,-8322000000,-5978000000,-7.633000e+09,-8273000000,...,-7722000000,-7.979000e+09,-7844000000,-9901000000,-8060000000,-8.456000e+09,-8278000000,-11064000000,-9269000000,-9.742000e+09
5,Cost of Goods & Services,None,None,None,NaN,None,None,None,-2.707800e+10,None,...,-7722000000,2.480100e+10,-7844000000,-9901000000,-8060000000,2.580500e+10,None,-11064000000,-9269000000,2.033300e+10
6,Cost of Financing Revenue,None,None,None,NaN,None,None,None,NaN,None,...,None,NaN,None,None,None,NaN,None,None,None,NaN
7,Cost of Other Revenue,None,None,None,NaN,None,None,None,NaN,None,...,None,NaN,None,None,None,NaN,None,None,None,NaN
8,Gross Profit,11840000000,15764000000,15702000000,1.415800e+10,13384000000,16197000000,14425000000,1.574900e+10,14928000000,...,12809000000,1.846900e+10,14084000000,15925000000,15152000000,1.714900e+10,16260000000,17854000000,17550000000,2.034300e+10
9,Other Operating Income,None,None,None,NaN,None,None,None,NaN,None,...,None,NaN,None,None,None,NaN,None,None,None,NaN
